## Localization

In this notebook we will understand the concept of localization. These notes are prepared while I took the [cs373](https://www.udacity.com/course/artificial-intelligence-for-robotics--cs373) course on Udacity

---

Informal definition of Localization is about knowing where the Robot/Autonomous vehicle is in the given terrain/map accurately. GPS has a way to tell us where we are, however, the accuracy if off by meters most of the times. For autonomous vehicles knowing the location to centimeter accuracy is crucial to stay within the lanes (in case of self driving cars) even if no markings are present.


Lets take a brief look at what exactly is the localization and what math is involved in it. We will look at the same diagrams as the ones in the lecture.

---

1. Assume that we have a map in one dimension (we can move right or left) and the robot has no clue where it is at the moment. The probability at this point is same for each location in the map. That is, the probability of Robot being present at a location is uniformly distributed over the entire map region and the probability distribution would look like the following.

![prior](Prior.png)

2. Assume the map has three doors and the robot can detect a door and non door area in front of it. Given that the robot senses there is a door in front of it (which can have an error in detection), the probability distribution of the robot's location changes to the following

![posterior](Posterior.png)

3. Notice how we have three peaks in front of the doors. At this stage the probability of the robot being in front of these three doors is same. Since we have errors in the sensors, there is a small residual probability in other regions as well. 

4. The robot can move in the right or left direction and assuming it knows where it moves, the probability distribution for the robot's current location shifts as the robot moves as we see below

![posterior2](Posterior2.png)

5. Now with the new information that the root sees another door in front of it, the probability distribution changes as follows. 

![posterior3](Posterior3.png)

Notice how the posterior of the previous state becomes the prior for this new state. Thus as and how the robot moves and sees/senses new information and **given that the map is available with the robot**, the robot can continue to accurately detect its current location.

---

Lets look at a problem as follows write a program which will compute the probability of the robot being in front of the given colored grid, given it sees a grid's color.

The grid looks as follows.

![grid](Grids.png)


- Lets assume we have equal likelyhood of being in front of any grid initially
- Assume the probability the robot will sense the grid as red, given the grid if red is 0.6, we call this ``pHit``
- Assume the probability the robot will not sense the grid as red, given the grid if red is 0.2, we call this ``pMiss``
- Now given the fact the robot sees red, lets compute the probability the robot is standing in front of a given cell in the grid


In [4]:
p = [0.2, 0.2, 0.2, 0.2, 0.2]
world = ['green', 'red', 'red', 'green', 'green']
Z = 'red'
pHit = 0.6
pMiss = 0.2

def sense(p, Z):
    q = [p[i] * pHit if Z == world[i] else p[i] * pMiss for i in range(len(p))]        
    s = sum(q)
    return [q[i] / s for i in range(len(q))]

print(sense(p,Z))

[0.1111111111111111, 0.3333333333333332, 0.3333333333333332, 0.1111111111111111, 0.1111111111111111]


---

As we see above, the probability we are in front of a cell in red is 3 times as much as the probability we are in front of any other cell

Let us now see how we would have solved the above problem using [Bayes Rule](https://en.wikipedia.org/wiki/Bayes%27_theorem) and get the same answer.

Bayes Theorem states that $P(A \mid B)$ = $P(B \mid A) \cdot P(A) / P(B)$


Looking at the above diagram, we see there are 2 reds out of 5. Thus we can say the following

$P(R) = 2 / 5 = 0.4$

$P(G) = 3 / 5 = 0.6$

We now know two things, the probability the sensir will sense red, given its red, and, the probability the Sensor will sense red when the color ahead is not Red (false positives). Lets write them down

$P(S_r \mid R) = 0.6$

$P(S_r \mid \neg R) = 0.2$


As per Bayes Rule the color is Red, given the sensor senses Red

$P(R \mid S_r) = P(S_r | R) \cdot P(R) / P(S_r)$

$P(S_r) = P(S_r \mid R) \cdot P(R) + P(S_r \mid \neg R) \cdot P(\neg R) = 0.6 \cdot 0.4 + 0.2 \cdot 0.6 = 0.36$

$P(S_r \mid R) \cdot P(R) = 0.4 \cdot 0.6 = 0.24$


Thus $P(R \mid S_r) = 0.24 / 0.36 = 0.66667$

Since its equaly likely to be in front of any Reds, the probability that we are in front of one of these is $0.666 / 2 = 0.333$

Similarly, $P(G) = 1 - P(R) = 1 - 0.666  = 0.333$

Since ots equaly likely to be in front of any of the three green slots, the probability we are in front of one of them is $0.666 / 3 = 0.111$


These are the numbers we got in the program above.

---

## Robot Motion

Lets talk about Robot motion. Lets assume, we have cells and the robot moves exactly one cell at a time to the right. Assuming the cells are cyclic (such that what exits the right enters from left), each time a robot moves to its right by exactly one cell with 100% proability, the probabilities also shift right with the robot

Thus say we have the following set of probablities for the 5 slots

$\mid 0.11 \mid 0.33 \mid 0.33 \mid 0.11 \mid 0.11 \mid$

Assuming the robot moves to its right, the new set of probabilities where the robot is right now becomes

$\mid 0.11 \mid 0.11 \mid 0.33 \mid 0.33 \mid 0.11 \mid$

Notice how the probabilities are shifted to right.


Lets now code this function ``move`` which takes two arguments, 

- ``p:`` The prior probabilities
- ``u:`` The blocks to shift by, if positive, shift right else left


In [9]:
def move(p, U):
    l = len(p)
    return [p[(i - U) % l] for i in range(l)]


Lets test this function for a sample input

In [10]:
s_p = [0, 1, 0, 0, 0, 0]
print('p is ', s_p)
print('Moving p with u = 1 gives ', move(s_p, 1))
print('Moving p with u = -1 gives ', move(s_p, -1))

p is  [0, 1, 0, 0, 0, 0]
Moving p with u = 1 gives  [0, 0, 1, 0, 0, 0]
Moving p with u = -1 gives  [1, 0, 0, 0, 0, 0]


We will not look at inaccurate motion for robots.